In [1]:
from matplotlib import pyplot as plt
from mtcnn.mtcnn import MTCNN
from matplotlib.patches import Rectangle
from numpy import asarray
from PIL import Image
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
from scipy.spatial.distance import cosine
import cv2
import os


Using TensorFlow backend.


In [ ]:
detector = MTCNN()
def detectface(image):
    faces = detector.detect_faces(image)
    return faces

In [ ]:
def extract_face_from_image(image_path, required_size=(224, 224)):
  # load image and detect faces
    image = plt.imread(image_path)
    detector = MTCNN()
    faces = detector.detect_faces(image)

    face_images = []

    for face in faces:
        # extract the bounding box from the requested face
        x1, y1, width, height = face['box']
        x2, y2 = x1 + width, y1 + height

        # extract the face
        face_boundary = image[y1:y2, x1:x2]

        # resize pixels to the model size
        face_image = Image.fromarray(face_boundary)
        face_image = face_image.resize(required_size)
        face_array = asarray(face_image)
        face_images.append(face_array)

    return face_images

In [ ]:
def get_model_scores(faces):
    samples = asarray(faces, 'float32')

    # prepare the data for the model
    samples = preprocess_input(samples, version=2)

    # create a vggface model object
    model = VGGFace(model='resnet50',
      include_top=False,
      input_shape=(224, 224, 3),
      pooling='avg')

    # perform prediction
    return model.predict(samples)

In [ ]:
def load_images_from_folder(folder):
    images = []
    filenames = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
            filenames.append(os.path.splitext(os.path.basename(filename))[0])
    
    return images,filenames

In [ ]:
files=[]
fold_img,filenames=load_images_from_folder("face recognition/extracted faces")

f1=faces1 = extract_face_from_image('img1.jpg')
#faces2 = extract_face_from_image('img1.jpg')

model_scores_face1 = get_model_scores(faces1)

for i,faces2 in enumerate(fold_img):
    faces2 = extract_face_from_image(faces2) #if you have already extracted faces then don't use this
    #faces22=np.reshape(faces2,(1,faces2.shape[0],faces2.shape[1],faces2.shape[2]))
    model_scores_face2 = get_model_scores(faces2)
    for idx, face_score_1 in enumerate(model_scores_face1):
        for idy, face_score_2 in enumerate(model_scores_face2):
            score = cosine(face_score_1, face_score_2)
            if score <= 0.3:
                # Printing the IDs of faces and score
                print(score)
                f1.pop(idx)
                filenames.pop(i)
                
                # Displaying each matched pair of faces
print("Suspicious people are")
for i in f1:
    plt.imshow(f1[idx])
    plt.show()
print("Absent Students are")
for i in filenames:
    print(i)


In [ ]:
get_model_scores(extract_face_from_image(faces2))